# Problem
This problem was asked by Google.

An XOR linked list is a more memory efficient doubly linked list. Instead of each node holding next and prev fields, it holds a field named both, which is an XOR of the next node and the previous node. Implement an XOR linked list; it has an add(element) which adds the element to the end, and a get(index) which returns the node at index.

If using a language that has no pointers (such as Python), you can assume you have access to get_pointer and dereference_pointer functions that converts between nodes and memory addresses.

In [113]:
import ctypes

class Node:
    def __init__(self, val):
        self.val = val
        self.both = 0
    
    def __repr__(self):
        return 'Node <value: %(value)d, both: %(both)d>' % {
                    'value': self.val,
                    'both': self.both
                }         

class XORLinkedList:
    def __init__(self):
        self.head = self.tail = None
        self.__nodes = []
    
    def __repr__(self):
        if self.head:
            return '''
                Head <value: %(hvalue)d, both: %(hboth)d>,
                Tail <value: %(tvalue)d, both: %(tboth)d>
            ''' % {
                    'hvalue': self.head.val,
                    'hboth': self.head.both,
                    'tvalue': self.tail.val,
                    'tboth': self.tail.both
            }
        return 'Empty doubly linked list'
        
    def get(self, index):
        prev_id = 0
        node = self.head
        for i in range(index):
            next_id = prev_id ^ node.both
            if next_id:
                prev_id = id(node)
                node = _get_obj(next_id)
            else:
                raise IndexError('Linked list index out of range')
        return node     

    def add(self, node):
        if self.head is None:
            self.head = self.tail = node
        else:
            self.tail.both = id(node) ^ self.tail.both          
            node.both = id(self.tail)
            self.tail = node

        # Without this line, Python thinks there is no way to reach nodes between head and tail.
        self.__nodes.append(node)
    
    def insert(self, i, node): # plus
        if not self.head:
            return self.__repr__()
        
        if i == 0:
            node.both = id(self.head) ^ 0
            self.head.both = self.head.both ^ id(node)
            self.head = node
        else:
            try:
                ith_prev = self.get(i-1)
                ith = self.get(i)
            except IndexError as e:
                return str(e)

            ith_prev.both = ith_prev.both ^ id(ith) # removing the pointer to ith node
            ith_prev.both = ith_prev.both ^ id(node) # adding the pointer to the new node

            ith.both = id(ith_prev) ^ ith.both # removing the pointer to ith_prev node
            ith.both = ith.both ^ id(node) # adding the pointer to the new node

            node.both = id(ith_prev) ^ id(ith) # pointing the new node to ith_prev and ith nodes
        
        self.__nodes.insert(i, node)

In [114]:
def _get_obj(id):
    return ctypes.cast(id, ctypes.py_object).value   

In [115]:
xorll = XORLinkedList()
print(xorll)

Empty doubly linked list


In [116]:
xorll.add(Node(10))
xorll.add(Node(20))
xorll.add(Node(30))

In [117]:
xorll


                Head <value: 10, both: 4470160912>,
                Tail <value: 30, both: 4470160912>
            

In [118]:
xorll.insert(1, Node(15))

In [119]:
for i in range(4):
    print(xorll.get(i))

Node <value: 10, both: 4470160128>
Node <value: 15, both: 664>
Node <value: 20, both: 1064>
Node <value: 30, both: 4470160912>


In [120]:
xorll.insert(3, Node(25))

In [121]:
for i in range(5):
    print(xorll.get(i))

Node <value: 10, both: 4470160128>
Node <value: 15, both: 664>
Node <value: 20, both: 2600>
Node <value: 25, both: 312>
Node <value: 30, both: 4470157608>


In [122]:
xorll.insert(0, Node(5))

In [123]:
xorll


                Head <value: 5, both: 4470160520>,
                Tail <value: 30, both: 4470157608>
            

In [124]:
for i in range(6):
    print(xorll.get(i))

Node <value: 5, both: 4470160520>
Node <value: 10, both: 3488>
Node <value: 15, both: 664>
Node <value: 20, both: 2600>
Node <value: 25, both: 312>
Node <value: 30, both: 4470157608>
